In [ ]:
!wandb sync --sync-all --include-synced

In [8]:
!git clone https://github.com/CreatlV/thesis.git

Cloning into 'thesis'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 27 (delta 10), reused 25 (delta 8), pack-reused 0
Receiving objects: 100% (27/27), 32.83 KiB | 1.06 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [10]:
import sys
sys.path.append("/src")

# Prerequisites

In [2]:
# mex_path = '/content/drive/MyDrive/MEX'
mex_path = '/app/MEX'

In [3]:
## Categories sample scrape
# {1: 'title', 2: 'main-photo', 3: 'breadcrumb', 4: 'price', 5: 'image-slider', 6: 'description'}

## Categories CoVA
categories = {0: 'price', 1: 'title', 2: 'image'}

## Categories Apify
# images, longDescription, name, price, shortDescription, specification, category
# categories = {0: 'category', 1: 'image', 2: 'short-description', 3: 'name', 4: 'price', 5: 'long-description', 6: 'specification'}
categories = {0: 'category', 1: 'image', 2: 'price', 3: 'name'}

In [4]:
# id2label = {1: 'title', 2: 'main-photo', 3: 'breadcrumb', 4: 'price', 5: 'image-slider', 6: 'description'}
id2label = categories
label2id = {v: k for k, v in id2label.items()}

In [1]:
!nvidia-smi

Thu May 25 17:21:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:01:00.0 Off |                  N/A |
|  0%   29C    P8               22W / 350W|    579MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
%pip install datasets[vision] \
   git+https://github.com/CreatlV/transformers.git \
   timm evaluate matplotlib \
   torchmetrics[detection] ipywidgets wandb \
   python-dotenv scipy


  Cloning https://github.com/CreatlV/transformers.git to /tmp/pip-req-build-jo9gb49i
  Running command git clone --filter=blob:none --quiet https://github.com/CreatlV/transformers.git /tmp/pip-req-build-jo9gb49i
  Resolved https://github.com/CreatlV/transformers.git to commit 527ab894e59b6582578008e3b47648a65063f73d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 19.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import datasets
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from transformers import ConditionalDetrImageProcessor, ConditionalDetrForObjectDetection
from transformers import DeformableDetrImageProcessor, DeformableDetrForObjectDetection
from huggingface_hub import notebook_login
from transformers import TrainingArguments

import random

import datetime
from evaluate import load
import os
import wandb
from dotenv import load_dotenv

from display_utils import show_image_bbox_dataset, plot_results_all, detr_plot_results1
import dataset_utils
from evaluation import run_evaluation_cond
from CustomTrainer import CustomCDetrTrainer, get_collate_fn
from torch.utils.data import DataLoader
from datasets import concatenate_datasets, dataset_dict


ModuleNotFoundError: No module named 'display_utils'

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
load_dotenv()

# Data preparation

### CoVA - Load from source

In [ ]:
%%capture 
#Prevent output

# Set the path to the zip file
zip_path = mex_path + '/imgs.zip'

# Set the path to the folder where the contents of the zip file will be extracted
extract_path = '/content/images'

# Unzip the file
!unzip "{zip_path}" -d "{extract_path}"

In [ ]:
metadata_path = mex_path + '/covametadata.jsonl' 
data_path = extract_path + '/imgs' + '/metadata.jsonl'

print(metadata_path, data_path)

# Copy metadata file
! cp {metadata_path} {data_path}

In [ ]:
dataset = datasets.load_dataset("imagefolder", data_dir="./images/imgs")

In [ ]:
## Filter out data points without annotations

# print("From: ", len(dataset))
# keep = [i for i in range(len(dataset)) if len(dataset[i]["objects"]["bbox"]) > 0]
# dataset = dataset.select(keep)
# print("To: ", len(dataset))

In [ ]:
## Make sure it contains features: ['image', 'image_id', 'height', 'width', 'objects', 'annotations'],
dataset

In [ ]:
dataset = dataset.map(dataset_utils.transform_rgb, batched=True)

In [ ]:
## Folds 1..5 are available

path = mex_path + "/splits/Fold-1"

# Read the image_ids from the text files
with open(path + "/train_imgs.txt", "r") as f:
    train_image_ids = set([line.strip() for line in f.readlines()])

with open(path + "/val_imgs.txt", "r") as f:
    val_image_ids = set([line.strip() for line in f.readlines()])

with open(path + "/test_imgs.txt", "r") as f:
    test_image_ids = set([line.strip() for line in f.readlines()])

# Define a function to filter the dataset based on image_ids
def filter_dataset_by_ids(example, image_ids):
    return example["image_id"] in image_ids

In [ ]:
train_dataset = dataset.filter(lambda x: filter_dataset_by_ids(x, train_image_ids))
val_dataset = dataset.filter(lambda x: filter_dataset_by_ids(x, val_image_ids))
test_dataset = dataset.filter(lambda x: filter_dataset_by_ids(x, test_image_ids))

In [ ]:
dataset["train"] = train_dataset
dataset["val"] = val_dataset
dataset["test"] = test_dataset

In [ ]:
dataset.save_to_disk(mex_path + "/cova-dataset")

### CoVa - Load dataset drive

In [ ]:
dataset = datasets.load_from_disk("/content/drive/MyDrive/MEX/cova-dataset")

### Load from HUB

In [ ]:
# Get huggingface token from env variable
token =  os.getenv('HUGGINGFACE_TOKEN')

In [ ]:
from huggingface_hub import HfApi, HfFolder
api=HfApi()
api.set_access_token(token)
folder = HfFolder()
folder.save_token(token)

In [ ]:
dataset = datasets.load_dataset("CreatlV/cova-coco-v2")

In [ ]:
dataset = datasets.load_dataset("CreatlV/apify-v1")

### Pre-process Apify

In [ ]:
!rm -rf "/root/.cache"

In [ ]:
# ## Filter all annotation to remove the categories of short description, long description and specifications

# def filter_annotations(example):
#     mew_annotations = []
#     new_objects = {"bbox": [], "categories": []}
    
#     for annotation in example["annotations"]:
#         new_category_id = annotation["category_id"]
#         if new_category_id not in [2, 5, 6, None]:
#             mew_annotations.append(annotation)
#             new_objects["bbox"].append(annotation["bbox"])
#             if new_category_id not in new_objects["categories"]:
#                 new_objects["categories"].append(new_category_id)

#     example["annotations"] = mew_annotations
#     example["objects"] = new_objects

#     return example

# dataset_filter_annotations = dataset.map(filter_annotations)


In [ ]:
## Re-create annotations and map all categories of id 4 to id 2.

def filter_map_annotations(example):
    new_annotations = []
    new_objects = {"bbox": [], "categories": []}
    
    for annotation in example["annotations"]:
        new_category_id = annotation["category_id"]
        # Filter out specification, and descriptions
        if new_category_id not in [2, 5, 6, None]:
          # Map price to id 2
          if new_category_id == 4:
            new_category_id = 2
          annotation["category_id"] = new_category_id
          new_annotations.append(annotation)
          new_objects["bbox"].append(annotation["bbox"])
          if new_category_id not in new_objects["categories"]:
              new_objects["categories"].append(new_category_id)

    example["annotations"] = new_annotations
    example["objects"] = new_objects

    return example

dataset_filter_map_annotations = dataset.map(filter_map_annotations)


In [ ]:
## Filter out data points without annotations

def filter_no_annotations(example):
    objects = example["objects"]
    if len(objects["categories"]) == 0:
        return False
    return True

print("From: ", len(dataset["train"]))
dataset_filter_no_annotations = dataset_filter_annotations.filter(filter_no_annotations, batch_size=50)
print("To: ", len(dataset_filter_no_annotations["train"]))

In [ ]:
## Add a new column called website domain and create it from the file_name
def add_website_domain(example):
    example['website_domain'] = example['file_name'].split('-')[2]
    print(example['website_domain'])
    return example

dataset_filter_no_annotations = dataset_filter_no_annotations.map(add_website_domain)

In [ ]:
## Crop all the images to a max length. Make sure >98% of remaining labels are still kept
from PIL import Image
import numpy as np
import io

def crop_and_update_bboxes(example):
    # Define the maximum height
    max_height = 1280

    # Open image using PIL
    img = example['image']

    # Check if height is greater than max_height
    if img.size[1] > max_height:
        # Crop the image
        img = img.crop((0, 0, img.size[0], max_height))

        # Convert image back to byte array
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        img_byte_arr = img_byte_arr.getvalue()

        # Update the example with the new image
        example['image'] = Image.open(io.BytesIO(img_byte_arr))

    # Update the height and width of the object
    example['height'] = img.size[1]
    example['width'] = img.size[0]

    # Update bounding boxes
    new_bboxes = []
    new_categories = []
    new_annotations = []
    for bbox, category, annotation in zip(example['objects']['bbox'], example['objects']['categories'], example['annotations']):
        x, y, width, height = bbox
        # Check if bbox is within the cropped image
        if y + height <= max_height:
            new_bboxes.append(bbox)
            new_categories.append(category)
            new_annotations.append(annotation)
        else:
            # Adjust the bbox and annotation if necessary
            if y < max_height:
                new_height = max_height - y
                bbox[3] = new_height
                annotation['bbox'][3] = new_height
                annotation['area'] = width * new_height
                new_bboxes.append(bbox)
                new_categories.append(category)
                new_annotations.append(annotation)

    # Update objects and annotations in the example
    example['objects']['bbox'] = new_bboxes
    example['objects']['categories'] = new_categories
    example['annotations'] = new_annotations

    return example

# Apply the function to the dataset
dataset_cropped_images = dataset_filter_no_annotations.map(crop_and_update_bboxes)


In [ ]:
## Rename image_id to file_name and hash the file_name to create a unique integer image_id
def rename_image_id(example):
    if 'file_name' not in example:
        example['file_name'] = example['image_id']

    example['image_id'] = hash(example['file_name'])
    for annotation in example['annotations']:
        annotation['image_id'] = example['image_id']

    return example

dataset_renamed_image_id = dataset.map(rename_image_id)

In [ ]:
dataset = dataset_renamed_image_id

In [ ]:
## Save the dataset to disk
dataset.save_to_disk(mex_path + "/apify-dataset-processed-4")

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

In [ ]:
## Combine train and test dataset
dataset_combined = concatenate_datasets([dataset["train"], dataset["test"]])

In [ ]:
unique_domains = set(dataset_combined['website_domain'])

In [ ]:
## First get 2 unique domains for test
random.seed(42)
test_domains = random.sample(list(unique_domains), 2)
print(test_domains)

In [ ]:
dataset_train = dataset.filter(lambda x: x['website_domain'] not in test_domains, batch_size=500)
dataset_test = dataset.filter(lambda x: x['website_domain'] in test_domains, batch_size=500)

In [ ]:
dataset = {"train": dataset_train, "test": dataset_test}
dataset = dataset_dict.DatasetDict(dataset)

In [ ]:
## Save the new dataset split to disk
dataset.save_to_disk(mex_path + "/apify-dataset-processed-website-split-" + test_domains[0] + "-" + test_domains[1])

In [ ]:
dataset

### Choose feature extraction

In [ ]:
feature_extractor = DetrImageProcessor()

In [ ]:
feature_extractor = ConditionalDetrImageProcessor()

In [ ]:
feature_extractor = DeformableDetrImageProcessor()

### General

In [ ]:
dataset

In [13]:
from display_utils import show_image_bbox_dataset


show_image_bbox_dataset(random.randint(0, len(dataset["train"]) - 1), dataset["train"], categories)

ModuleNotFoundError: No module named 'display_utils'

In [ ]:
def transform(example_batch):
    images = example_batch["image"]
    all_annotations = example_batch["annotations"]
    targets = [
        {"image_id": annotations[0]["image_id"], "annotations": annotations} for annotations in all_annotations
    ]
    return feature_extractor(images=images, annotations=targets, return_tensors="pt")

In [ ]:
dataset_train = dataset["train"].with_transform(transform)
dataset_test = dataset["test"].with_transform(transform)
if "val" in dataset:
    dataset_val = dataset["val"].with_transform(transform)
else:
    dataset_val = dataset_test

In [ ]:
dataset_train

In [ ]:
dataset_test

In [ ]:
dataset_val

# Fine-tuning

In [ ]:
now = datetime.datetime.now()
date_time_str = now.strftime("%Y-%m-%d_%H:%M:%S")

### Deformable DETR

In [ ]:
output_path = mex_path + "/deformable-detr-resnet-50_fine_tuned_cova_" + date_time_str

In [ ]:
model = DeformableDetrForObjectDetection.from_pretrained(
    "SenseTime/deformable-detr",
    num_labels=len(categories.keys()),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    num_queries=20
)

### Conditional DETR

In [ ]:
output_path = mex_path + "/conditional-detr-resnet-50_fine_tuned_cova_" + date_time_str

In [ ]:
## Use pretrained model
model = ConditionalDetrForObjectDetection.from_pretrained(
    "microsoft/conditional-detr-resnet-50",
    num_labels=len(categories.keys()),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    num_queries=20,
    backbone="resnet101",
    use_pretrained_backbone=True,
    use_timm_backbone=True,    
)


In [ ]:
## Instantiate DETR with randomly initialized weights for Transformer, but pre-trained weights for backbone
from transformers import ConditionalDetrConfig
config = ConditionalDetrConfig(
    num_labels=len(categories.keys()),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    num_queries=20,
    )

model = ConditionalDetrForObjectDetection(config)

### Common

In [ ]:
fine_tuning_training_args = TrainingArguments(
    output_dir=output_path,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    logging_steps=50,
    max_grad_norm=0.1,
    weight_decay=10e-4,
    evaluation_strategy="steps",
    eval_steps=1000,
    # learning_rate=1e-4,
    remove_unused_columns=False,
)

In [ ]:
# Create the custom optimizer with grouped parameters

retrain_training_args = TrainingArguments(
    output_dir=output_path,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=200,
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=1000,
    weight_decay=10e-4,
    max_grad_norm=0.1, # Gradient clipping
    # learning_rate=1e-4,
    remove_unused_columns=False
)

In [ ]:
from typing import Literal, Union


training_mode: Union[Literal["retrain"], Literal["finetune"]] = "finetune"

if training_mode == "retrain":
    training_args = retrain_training_args
elif training_mode == "finetune":
    training_args = fine_tuning_training_args

# training_args.report_to = []
training_args.report_to = ["wandb"]

print(dataset_train)

trainer = CustomCDetrTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub("conditional-detr-resnet-50_fine_tuned_cova_v2_40_epochs")

In [ ]:
trainer.push_to_hub()

# Model inference and evaluation

In [ ]:
### Download folder from google drive

# model_checkpoint_inference = output_path + "/checkpoint-10400"
model_checkpoint_inference_path = "/conditional-detr-resnet-50_fine_tuned_cova_2023-04-22_14:57:20"
model_checkpoint = "/checkpoint-31500"
model_checkpoint_inference = mex_path + model_checkpoint_inference_path + model_checkpoint



In [ ]:
huggingface_link = "CreatlV" + model_checkpoint_inference_path.replace(":", ".")
print(huggingface_link)

In [ ]:
## COND DETR model_checkpoint
model_checkpoint_inference = "CreatlV/conditional-detr-resnet-50_fine_tuned_cova_v2"

In [ ]:
### Conditional DETR
model = ConditionalDetrForObjectDetection.from_pretrained(
    model_checkpoint_inference,
    num_labels=len(categories.keys()),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    num_queries=20,
)

In [ ]:
### DETR on huggingface
model = DetrForObjectDetection.from_pretrained(
    huggingface_link,
    num_labels=len(categories.keys()),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    num_queries=100,
)

In [ ]:
dataset

## Examples from test dataset

In [ ]:
test_image_index = random.randint(0, len(dataset_test) - 1)
# test_image_index = 0

# print(outputs)
outputs = model(
    dataset_test[test_image_index]["pixel_values"].unsqueeze(-4),
    dataset_test[test_image_index]["pixel_mask"].unsqueeze(-3),
    return_dict=True,
)

print(outputs.logits.size())

# rescale bounding boxes
target_sizes = torch.tensor(
    dataset["test"][test_image_index]["image"].size[::-1]
).unsqueeze(0)

In [ ]:
## DETR

probas = outputs.logits.softmax(-1)[0, :, :]
postprocessed_outputs = feature_extractor.post_process_object_detection(
    outputs, 0, target_sizes
)
bboxes_scaled = postprocessed_outputs[0]["boxes"]
detr_plot_results1(
    dataset["test"][test_image_index]["image"], probas, bboxes_scaled, model
)

In [ ]:
## COND DETR

postprocessed_outputs = feature_extractor.post_process_object_detection(
        outputs, 0.0, target_sizes
    )
plot_results_all(
    dataset["test"][test_image_index]["image"],
    postprocessed_outputs[0]["boxes"],
    postprocessed_outputs[0]["labels"],
    postprocessed_outputs[0]["scores"],
    model=model,
)

In [ ]:
dataset["test"][0]

In [ ]:
show_image_bbox_dataset(0, dataset["test"], categories=categories)

## Test upload example

In [ ]:
single = datasets.load_dataset("imagefolder", data_dir="/app/single")

In [ ]:
single

In [ ]:
single = single.map(dataset_utils.transform_rgb, batched=True)

In [ ]:
def transform_test(example_batch):
    images = example_batch["image"]
    return feature_extractor(images=images, return_tensors="pt")

In [ ]:
single["train"][0]

In [ ]:
dataset_single = single["train"].with_transform(transform_test)

In [ ]:
dataset_single[0]

In [ ]:
outputs = model(dataset_single[0]['pixel_values'].unsqueeze(-4), dataset_single[0]['pixel_mask'].unsqueeze(-3))

In [ ]:
target_sizes = torch.tensor(single["train"][0]["image"].size[::-1]).unsqueeze(0)
postprocessed_outputs = feature_extractor.post_process_object_detection(outputs, 0.8, target_sizes)
plot_results_all(
  single["train"][0]["image"],
  postprocessed_outputs[0]['boxes'],
  postprocessed_outputs[0]['labels'],
  postprocessed_outputs[0]['scores'],
  model=model
  )

## mAP calculation

### Custom EVAL - Experiment

In [ ]:
## CURRENT DETR EVALUATION CAN ONLY WORK WITH BATCH SIZE 1
## Conditional DETR can work with batch size > 1
collate_fn = get_collate_fn(feature_extractor)


test_dataloader = DataLoader(dataset_test, collate_fn=collate_fn, batch_size=6)
# test_dataloader = DataLoader(dataset_test, collate_fn=collate_fn, batch_size=1)

### BOTH

In [ ]:
device = torch.device("cuda")

run_evaluation_cond(model, feature_extractor, test_dataloader, device, plot=False)


### Trainer evaluation

In [ ]:
now = datetime.datetime.now()
date_time_str = now.strftime("%Y-%m-%d_%H:%M:%S")

output_path = mex_path + "/EVALUATE-conditional-detr-resnet-50_fine_tuned_cova_" + date_time_str

training_args = TrainingArguments(
    output_dir=output_path,
    per_device_eval_batch_size=16,
    remove_unused_columns=False,
    push_to_hub=False,
)

In [ ]:
training_args.report_to = []

trainer = CustomCDetrTrainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_test,
    tokenizer=feature_extractor,
)

In [ ]:
trainer.evaluate()